In [5]:
using Plots, ApproxFun
plotly();  # also works in GR;

# Laplace Equation $u_{xx} + u_{yy} = 0$, $u|_{\partial d} = \Re(e^{x+i y})$

In [6]:
d=Interval()^2
g=Fun((x,y)->real(exp(x+im*y)),∂(d))  # boundary data
Δ=Laplacian(d)
u=[Dirichlet(d);Δ]\[g;0]
plot(u)

We solve with neumann on one edge
`ldirichlet/rdirichlet/lneumann/rneumann` commands specify left/right dirichet/neumann boundary conditions

In [7]:
dx=dy=Interval()
d=dx*dy

x,y=Fun(∂(d))
x,y=vec(x),vec(y)

g=[-exp(x[1])*sin(-1);exp(1)*cos(y[2]);
    exp(x[3])*cos(1);exp(-1)*cos(y[4])]


Δ=Laplacian(d)

B=[I⊗lneumann(dy);rneumann(dx)⊗I;I⊗rdirichlet(dy);ldirichlet(dx)⊗I]
u=linsolve([B;Δ],[g;0];tolerance=1E-10)

plot(u)

# Poisson equation $u_{xx} + u_{yy} = f(x,y)$

In [8]:
f=Fun((x,y)->exp(-10(x+.2)^2-20(y-.1)^2))  #default is [-1,1]^2
d=domain(f)
Δ=Laplacian(d)
u=[Dirichlet(d);Δ]\[0;f]
plot(u)

# Helmholtz $u_{xx} + u_{yy} + 100u=0, u|_{\partial d}=1$

In [11]:
d=Interval()^2

Δ=Laplacian(d)

@time u=linsolve([Dirichlet(d);Δ+100I],[ones(∂(d));0];tolerance=1E-5)
plot(u)

  5.909429 seconds (16.60 M allocations: 582.178 MB, 2.67% gc time)


In [12]:
d=Interval()^2

Δ=Laplacian(d)

@time u=[neumann(d);Δ+100I]\[ones(4);0]
plot(u)

  0.528989 seconds (3.93 M allocations: 118.659 MB, 8.43% gc time)


# Screened Poisson  $u_{xx} + u_{yy} - 100u = 0, \partial u(\partial d) = 1$

In [13]:
d=Interval()^2
Δ=Laplacian(d)

@time u=[neumann(d);Δ-100.0I]\[ones(4);0.0]
plot(u)

  0.493118 seconds (3.01 M allocations: 94.000 MB, 6.98% gc time)


# Convection $u_t + u_x = 0,u(x,0)=e^{-20x^2}, u(-1,t) = 0$

In [14]:
dx=Interval();dt=Interval(0,2.)
d=dx*dt
Dx=Derivative(d,[1,0]);Dt=Derivative(d,[0,1])
u0=Fun(x->exp(-20x^2),dx)
@time u=linsolve([I⊗ldirichlet(dt);ldirichlet(dx)⊗I;Dt+Dx],[u0;0;0];
            tolerance=1E-3)
plot(u)

  4.480527 seconds (5.80 M allocations: 234.452 MB, 6.97% gc time)


## Piecewise PDE $$u_t+\begin{cases} 0.5 & 0≤x≤0.5\cr                     1 & \hbox{otherwise}\end{cases} u_{xx}$$

We can solve on piecewise domains. Here is convection with two different speeds, imposing continuity at the singularities.

In [15]:
a=Fun([1,0.5,1],[-1.,0.,0.5,1.])
s=space(a)
dt=Interval(0,2.)
Dx=Derivative(s);Dt=Derivative(dt)
Bx=[ldirichlet(s);continuity(s,0)]
@time u=linsolve([I⊗ldirichlet(dt);Bx⊗I;I⊗Dt+(a*Dx)⊗I],
    [Fun(x->exp(-20(x+0.5)^2),s);zeros(length(Bx));0.0];tolerance=1E-1)

plot(u)

 41.845508 seconds (96.37 M allocations: 3.433 GB, 4.36% gc time)


# Convection $u_t - x u_x = 0, u(x,0)=e^{-20x^2}, u(-1,t)=u(1,t)=0$

In [16]:
dx=Interval();dt=Interval(0,2.)
d=dx*dt
Dx=Derivative(d,[1,0]);Dt=Derivative(d,[0,1])
x,t=Fun(identity,d)

#timedirichlet is [u(x,0), u(-1,t), u(1,t)]
u0 = Fun(x->exp(-20x^2),dx)
@time u=linsolve([timedirichlet(d);Dt-x*Dx],[u0;0;0;0];tolerance=1E-3)
plot(u)

 24.139364 seconds (40.57 M allocations: 1.906 GB, 4.85% gc time)


# Convection $u_t + x u_x = 0, u(x,0)=e^{-20x^2}$

In [17]:
dx=Interval();dt=Interval(0,2.)
d=dx*dt
Dx=Derivative(d,[1,0]);Dt=Derivative(d,[0,1])
x,y=Fun(d)
u0=Fun(x->exp(-20x^2),dx)
@time u=linsolve([I⊗ldirichlet(dt);Dt+x*Dx],[u0;0];tolerance=1E-10)
plot(u)

  3.025921 seconds (4.80 M allocations: 194.231 MB, 2.69% gc time)


# Convection diffusion $u_t = \epsilon u_{xx} + (2+x) u_x, u(x,0)=e^{-20x^2}, u(-1,t)=u(1,t) = 0$

In [18]:
dx=Interval();dt=Interval(0,1.)
d=dx*dt
ε=.01
Dx=Derivative(d,[1,0]);Dt=Derivative(d,[0,1])
x,t=Fun(d)
V=2.0+x
# Parentheses are a hack to get rank 2 PDE
u0=Fun(x->exp(-20x^2),dx)
@time u=linsolve([timedirichlet(d);Dt-ε*Dx^2-V*Dx],[u0;0;0;0];
                    tolerance=1E-6)

plot(u)

  1.796031 seconds (6.70 M allocations: 234.144 MB, 4.70% gc time)


# Wave equation with left Dirichlet and right Neumann $u_{tt} = u_{xx}, u(x,0)=e^{-20(x-.1)^2},u_t(x,0)=0,u(-1,t)=0,u_x(1,t)=0$

In [19]:
dx=Interval(-1.,1.);dt=Interval(0,4.)
d=dx*dt
Dx=Derivative(d,[1,0]);Dt=Derivative(d,[0,1])
# need to specify both ic and its derivative
B=[I⊗ldirichlet(dt),I⊗lneumann(dt),ldirichlet(dx)⊗I,rneumann(dx)⊗I]

u0=Fun(x->exp(-20(x-.1)^2),dx)
@time u=linsolve([B;Dt^2-Dx^2],[u0;0;0;0;0];
                    tolerance=1E-4)

plot(u)

  4.368717 seconds (9.14 M allocations: 341.461 MB, 8.63% gc time)


# Linear KdV Dirichlet $u_t + u_{xxx} = 0, u(x,0)=e^{-10x^2},u(-1,t)=u(1,t)=u_x(1,t)=0$

In [20]:
dx=Interval(-5.0,1.);dt=Interval(0,.1);
d=dx*dt
Dx=Derivative(d,[1,0]);Dt=Derivative(d,[0,1])
B=[timedirichlet(d);rneumann(dx)⊗I]
u0=Fun(x->exp(-10x^2),dx)
@time u=linsolve([B;Dt+Dx^3],[u0;0;0;0;0];tolerance=1E-3)

plot(u)

  9.052907 seconds (18.54 M allocations: 852.469 MB, 6.56% gc time)


## Linear KdV Neumann $u_t + u_{xxx} = 0, u(x,0)=e^{-10x^2},u_x(-1,t)=u(1,t)=u_x(1,t)=0$

In [21]:
dx=Interval(-5.0,1.);dt=Interval(0,.1);
d=dx*dt
Dx=Derivative(d,[1,0]);Dt=Derivative(d,[0,1])
B=[I⊗ldirichlet(dt);neumann(dx)⊗I;rdirichlet(dx)⊗I]
u0=Fun(x->exp(-10x^2),dx)
@time u=linsolve([B;Dt+Dx^3],[u0;0;0;0;0];tolerance=1E-4)

plot(u)

  7.367974 seconds (17.63 M allocations: 824.534 MB, 8.13% gc time)


# Beamer equation $u_{tt} + u_{xxxx} = 0, u(x,0)=e^{-200(x-1/2)^2}, u_t(x,0)=u(\pm,t)=u_x(\pm 1,t) = 0$

In [22]:
dx=Interval(0.0,1.0);dt=Interval(0,0.03)
d=dx*dt
Dx=Derivative(d,[1,0]);Dt=Derivative(d,[0,1]);
x,t=Fun(d)

u0=Fun(x->exp(-200(x-.5)^2),dx)

@time u=linsolve(
    [timedirichlet(d);I⊗lneumann(dt);neumann(dx)⊗I;Dt^2+Dx^4],
    [u0;0;0;0;0;0;0];tolerance=1E-4)

plot(u)

 15.205960 seconds (36.97 M allocations: 1.404 GB, 6.64% gc time)


# Biharmonic equation $\Delta^2 u = f(x,y)$

In [23]:
S=WeightedJacobi(2.,2.)^2

Δ=Laplacian(S)

@time f=chop(Fun((x,y)->exp(-30(x^2+y^2)),rangespace(Δ^2)),1E-10)
@time u=linsolve(Δ^2,f;tolerance=1E-8)
plot(u)

  8.285714 seconds (8.97 M allocations: 483.044 MB, 5.95% gc time)
 12.354593 seconds (22.15 M allocations: 892.821 MB, 3.08% gc time)


# Schrödinger Dirichlet $i \epsilon u_t + .5 \epsilon^2 u_{xx} = x^2 u, u(x,0)=e^{-25(x-.5)^2}e^{-i/(5ϵ)log(2cosh(5(x-.5)))}$

In [24]:
dx=Interval(0.,1.);dt=Interval(0.0,0.54)
d=dx*dt

ϵ=0.0256
u0=Fun(x->exp(-25*(x-.5)^2)*exp(-1.0im/(5*ϵ)*log(2cosh(5*(x-.5)))),dx)

x,t=Fun(d)
V=x^2

Dt=Derivative(d,[0,1]);Dx=Derivative(d,[1,0])

L=1im*ϵ*Dt+.5*ϵ^2*Dx^2-V

@time u=linsolve([timedirichlet(d);L],[u0;0;0;0];tolerance=1E-2)

plot(real(u))

  5.530754 seconds (15.99 M allocations: 1.065 GB, 12.77% gc time)
